<h1>Pattern Mining</h1>

In [47]:
%pip install pandas
%pip install mlxtend
%pip install numpy
%pip install plotly

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=12505) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

df = pd.read_csv("./data/data.csv", sep=';')

/var/folders/vt/0x3f4crs5lb36gfgww7p5kb40000gn/T/ipykernel_12505/3220941569.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/data.csv", sep=';')


In [49]:
df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [50]:
print("#" * 50)
print(" " * 15, "Dataset Information")
print("#" * 50)
print("The Dataset has {} columns and {} rows.".format(df.shape[1], df.shape[0]))
print("The DataFrame has {} duplicated values with {} % and {} missing values.".format(df.duplicated().sum(),round((df.duplicated().sum())/len(df)*100,2),df.isnull().sum().sum()))
print(df.info())

##################################################
                Dataset Information
##################################################
The Dataset has 7 columns and 522064 rows.
The DataFrame has 5284 duplicated values with 1.01 % and 135496 missing values.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB
None


In [51]:
df.drop_duplicates(inplace=True)

In [52]:
print("#" * 30)
print(" " * 5, "Null Distribution")
print("#" * 30)
null = df.isnull().sum().to_frame().rename(columns = {0:'count'})
null['%'] = (null['count'] / len(df)) * 100.
null = null.sort_values(by = '%', ascending = False)
null.style.background_gradient(cmap = 'PRGn')

##############################
      Null Distribution
##############################


,count,%
CustomerID,133967,25.923410
Itemname,1455,0.281551
BillNo,0,0.000000
Quantity,0,0.000000
Date,0,0.000000
Price,0,0.000000
Country,0,0.000000


In [53]:
df = df.rename(columns={'Itemname': 'ItemName'})
df['ItemName'] = df['ItemName'].str.lower()

In [54]:
#Dropping Missing rows in CustomerID column
df.dropna(subset=['CustomerID'], inplace=True)

#Dropping data with negative or zero quantity
df=df.loc[df['Quantity']>0]

#Dropping data with zero price
df=df.loc[df['Price']>'0']

#Dropping Non-product data.
df=df.loc[(df['ItemName']!='postage')&(df['ItemName']!='dotcom postage')&(df['ItemName']!='adjust bad debt')&(df['ItemName']!='manual')]

In [55]:
df['Date'] = pd.to_datetime(df['Date'], format="%d.%m.%Y %H:%M")
df['Price'] = df['Price'].str.replace(',','.')
df['Price'] = df['Price'].astype('float')
df['CustomerID'] = df['CustomerID'].astype('int')

In [56]:
#Make Month and year columns separately
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Day'] = df['Date'].dt.day_name()

#Creating a Total price column
df['Total price']=df.Quantity*df.Price

In [57]:
df.sample(5)

,BillNo,ItemName,Quantity,Date,Price,CustomerID,Country,Month,Year,Day,Total price
9425,537209,pizza plate in box,3,2010-12-05 15:12:00,3.75,14606,United Kingdom,12,2010,Sunday,11.25
370,536401,english rose hot water bottle,1,2010-12-01 11:21:00,4.25,15862,United Kingdom,12,2010,Wednesday,4.25
218520,556797,glass apothecary bottle tonic,4,2011-06-14 14:49:00,3.95,17101,United Kingdom,6,2011,Tuesday,15.80
457234,576862,red hanging heart t-light holder,12,2011-11-16 16:06:00,2.95,14825,United Kingdom,11,2011,Wednesday,35.40
375033,570488,set/4 red mini rose candle in bowl,6,2011-10-10 17:12:00,1.63,14096,United Kingdom,10,2011,Monday,9.78


In [58]:
df

,BillNo,ItemName,Quantity,Date,Price,CustomerID,Country,Month,Year,Day,Total price
0,536365,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,12,2010,Wednesday,15.30
1,536365,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12,2010,Wednesday,20.34
2,536365,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,12,2010,Wednesday,22.00
3,536365,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12,2010,Wednesday,20.34
4,536365,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12,2010,Wednesday,20.34
...,...,...,...,...,...,...,...,...,...,...,...
522059,581587,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,12,2011,Friday,10.20
522060,581587,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12,2011,Friday,12.60
522061,581587,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,12,2011,Friday,16.60
522062,581587,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,12,2011,Friday,16.60


In [18]:
##Eclat (Equivalence Class Clustering and Bottom-Up Lattice Traversal): 

In [60]:

df.to_csv('./data/manipulated.csv', sep=',', encoding='utf-8')
df

,BillNo,ItemName,Quantity,Date,Price,CustomerID,Country,Month,Year,Day,Total price
0,536365,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,12,2010,Wednesday,15.30
1,536365,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12,2010,Wednesday,20.34
2,536365,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,12,2010,Wednesday,22.00
3,536365,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12,2010,Wednesday,20.34
4,536365,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12,2010,Wednesday,20.34
...,...,...,...,...,...,...,...,...,...,...,...
522059,581587,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,12,2011,Friday,10.20
522060,581587,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12,2011,Friday,12.60
522061,581587,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,12,2011,Friday,16.60
522062,581587,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,12,2011,Friday,16.60


In [61]:
transaction_list = df.groupby('BillNo')['ItemName'].apply(list).tolist()

In [62]:
transaction_list

[['white hanging heart t-light holder',
  'white metal lantern',
  'cream cupid hearts coat hanger',
  'knitted union flag hot water bottle',
  'red woolly hottie white heart.',
  'set 7 babushka nesting boxes',
  'glass star frosted t-light holder'],
 ['hand warmer union jack', 'hand warmer red polka dot'],
 ['assorted colour bird ornament',
  "poppy's playhouse bedroom",
  "poppy's playhouse kitchen",
  'feltcraft princess charlotte doll',
  'ivory knitted mug cosy',
  'box of 6 assorted colour teaspoons',
  'box of vintage jigsaw blocks',
  'box of vintage alphabet blocks',
  'home building block word',
  'love building block word',
  'recipe box with metal heart',
  'doormat new england'],
 ['jam making set with jars',
  'red coat rack paris fashion',
  'yellow coat rack paris fashion',
  'blue coat rack paris fashion'],
 ['bath building block word'],
 ['alarm clock bakelike pink',
  'alarm clock bakelike red',
  'alarm clock bakelike green',
  'panda and bunnies sticker sheet',
  

In [30]:
similarItemCount = df.groupby('ItemName').count()

In [31]:
similarItemCount

,BillNo,Quantity,Date,Price,CustomerID,Country,Month,Year,Day,Total price
ItemName,,,,,,,,,,
10 colour spaceboy pen,238,238,238,238,238,238,238,238,238,238
12 coloured party balloons,139,139,139,139,139,139,139,139,139,139
12 daisy pegs in wood box,62,62,62,62,62,62,62,62,62,62
12 egg house painted wood,57,57,57,57,57,57,57,57,57,57
12 hanging eggs hand painted,8,8,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...
zinc t-light holder stars small,232,232,232,232,232,232,232,232,232,232
zinc top 2 door wooden shelf,9,9,9,9,9,9,9,9,9,9
zinc willie winkie candle stick,187,187,187,187,187,187,187,187,187,187


In [79]:
te = TransactionEncoder()
te_ary = te.fit(transaction_list).transform(transaction_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,10 colour spaceboy pen,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 hanging eggs hand painted,12 ivory rose peg place settings,12 message cards with envelopes,12 pencil small tube woodland,12 pencils small tube red retrospot,12 pencils small tube skull,...,zinc star t-light holder,zinc sweetheart soap dish,zinc sweetheart wire letter rack,zinc t-light holder star large,zinc t-light holder stars large,zinc t-light holder stars small,zinc top 2 door wooden shelf,zinc willie winkie candle stick,zinc wire kitchen organiser,zinc wire sweetheart letter tray
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18048,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18049,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18050,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18051,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


<h1>FP Growth with mlxtend lib </h1>

In [66]:
trans_encoder = TransactionEncoder() # Instanciate the encoder
trans_encoder_matrix = trans_encoder.fit(transaction_list).transform(transaction_list)
trans_encoder_matrix = pd.DataFrame(trans_encoder_matrix, columns=trans_encoder.columns_)

In [67]:
trans_encoder_matrix.head()

,10 colour spaceboy pen,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 hanging eggs hand painted,12 ivory rose peg place settings,12 message cards with envelopes,12 pencil small tube woodland,12 pencils small tube red retrospot,12 pencils small tube skull,...,zinc star t-light holder,zinc sweetheart soap dish,zinc sweetheart wire letter rack,zinc t-light holder star large,zinc t-light holder stars large,zinc t-light holder stars small,zinc top 2 door wooden shelf,zinc willie winkie candle stick,zinc wire kitchen organiser,zinc wire sweetheart letter tray
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [81]:
def perform_rule_calculation(transact_items_matrix, rule_type="fpgrowth", min_support=0.001):
    start_time = 0
    total_execution = 0
    
    if(not rule_type=="fpgrowth"):
        start_time = time.time()
        rule_items = apriori(transact_items_matrix, 
                       min_support=min_support, 
                       use_colnames=True)
        total_execution = time.time() - start_time
        print("Computed Apriori!")
        
    else:
        start_time = time.time()
        rule_items = fpgrowth(transact_items_matrix, 
                       min_support=min_support, 
                       use_colnames=True)
        total_execution = time.time() - start_time
        print("Computed Fp Growth!")
    
    rule_items['number_of_items'] = rule_items['itemsets'].apply(lambda x: len(x))
    
    return rule_items, total_execution

In [82]:
def compute_association_rule(rule_matrix, metric="lift", min_thresh=1):
    rules = association_rules(rule_matrix, 
                              metric=metric, 
                              min_threshold=min_thresh)
    
    return rules

In [83]:
def plot_metrics_relationship(rule_matrix, col1, col2):
    fit = np.polyfit(rule_matrix[col1], rule_matrix[col2], 1)
    fit_funt = np.poly1d(fit)
    plt.plot(rule_matrix[col1], rule_matrix[col2], 'yo', rule_matrix[col1], 
    fit_funt(rule_matrix[col1]))
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.title('{} vs {}'.format(col1, col2))

In [84]:
def compare_time_exec(algo1=list, alg2=list):
    
    execution_times = [algo1[1], algo2[1]]
    algo_names = (algo1[0], algo2[0])
    y=np.arange(len(algo_names))
    
    plt.bar(y,execution_times,color=['orange', 'blue'])
    plt.xticks(y,algo_names)
    plt.xlabel('Algorithms')
    plt.ylabel('Time')
    plt.title("Execution Time (seconds) Comparison")
    plt.show()

In [85]:
fpgrowth_matrix, fp_growth_exec_time = perform_rule_calculation(trans_encoder_matrix) # Run the algorithm
print("Fp Growth execution took: {} seconds".format(fp_growth_exec_time))

Computed Fp Growth!
Fp Growth execution took: 18.65868091583252 seconds


In [86]:
fpgrowth_matrix.head()

,support,itemsets,number_of_items
0,0.106298,(white hanging heart t-light holder),1
1,0.017559,(knitted union flag hot water bottle),1
2,0.017393,(red woolly hottie white heart.),1
3,0.016230,(set 7 babushka nesting boxes),1
4,0.013128,(cream cupid hearts coat hanger),1


In [87]:
fpgrowth_matrix.tail()

,support,itemsets,number_of_items
266745,0.002437,"(classic chrome bicycle bell, bicycle puncture...",3
266746,0.001219,"(rabbit night light, bicycle puncture repair k...",3
266747,0.001108,"(classic chrome bicycle bell, rabbit night lig...",3
266748,0.001163,"(please one person metal sign, n0 singing meta...",2
266749,0.001329,"(please one person metal sign, gin and tonic d...",2


In [88]:
fp_growth_rule_lift = compute_association_rule(fpgrowth_matrix)

<h1>FP Growth with PySpark</h1>

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import concat_ws


spark = SparkSession.builder \
                    .appName("AssociationRuleMining") \
                    .getOrCreate()

data = spark.read \
            .format("csv") \
            .option("header", "true") \
            .load("file:////Users/sacithrangana/Desktop/quiz/Data Mining Assignment/DMProject/mba-project/Pattern Mining/data/manipulated.csv", header=True, inferSchema=True)


transactions_df = data.groupBy("BillNo").agg(collect_set("ItemName").alias("items"))

fp_growth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.01)


model = fp_growth.fit(transactions_df)

frequent_itemsets = model.freqItemsets
association_rules = model.associationRules

frequent_itemsets.show()
association_rules.show()

spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/23 04:56:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+----+
|               items|freq|
+--------------------+----+
|[ribbon reel chri...| 196|
|[wood s/3 cabinet...| 368|
|[airline bag vint...| 242|
|[heart ivory trel...| 219|
|[jumbo storage ba...| 469|
|[jumbo storage ba...| 196|
|[jumbo storage ba...| 247|
|[jumbo storage ba...| 185|
|[jumbo storage ba...| 207|
|[pack of 60 dinos...| 510|
|[pack of 60 dinos...| 257|
|[pack of 60 dinos...| 193|
|[pack of 60 dinos...| 214|
|[strawberry shopp...| 227|
|[vintage union ja...| 388|
|[piggy bank retro...| 248|
|[rabbit night light]| 787|
|[set of 36 doilie...| 185|
|[skull lunch box ...| 263|
|[pack of 72 retro...| 992|
+--------------------+----+
only showing top 20 rows



+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[lunch bag suki d...|[lunch bag cars b...| 0.5443349753694581|10.364758059487434|0.012243089025538752|
|[lunch bag suki d...|[lunch bag red re...| 0.6157635467980296| 8.779737585506503|0.013849648218935239|
|[lunch bag suki d...|[lunch bag spaceb...| 0.5344827586206896|  9.86497778718003|0.012021494654035788|
|[lunch bag suki d...|[lunch bag pink p...|  0.541871921182266|10.701674014508844| 0.01218769043266301|
|[lunch bag suki d...|[lunch bag woodland]|0.45320197044334976| 10.14981236783239|0.010193341089136335|
|[lunch bag suki d...|[lunch bag apple ...|0.49014778325123154|10.384574689516409| 0.01102431998227245|
|[pink regency tea...|[regency cakestan...| 0.5787037037037037| 